# Beer Recommender System
This notebook implements data preprocessing and modeling techniques to create a beer recommender system. I

In [1]:
#Imports
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


/Users/gustavnielsen/Documents/Comp_tools_project/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Preprocessing Functions
These functions clean the dataset by handling duplicates, missing values, and incorrect formats. They prepare the data for splitting and analysis.

In [2]:
def preprocess_data(df):
    df_filtered = df.drop_duplicates(["name", "reviewer", "review_text"]) # Remove duplicate entries
    print("Size after drop_duplicates: ", len(df_filtered))
    
    df_filtered['rating'] = pd.to_numeric(df_filtered['rating'], errors='coerce')  # Set erros to NaN
    df_filtered = df_filtered.dropna(subset=['rating'])  # Drop rows where 'rating' is NaN
    print("Size after drop rating NA: ", len(df_filtered))
    
    df_filtered['abv'] = pd.to_numeric(df_filtered['abv'].str.rstrip('%'), errors='coerce') 
    df_filtered = df_filtered.dropna(subset=['abv'])
    print("Size after drop abv NA: ", len(df_filtered))
    
    return df_filtered

def create_test_train(df, reviewer_col="reviewer", random_state=7, test_size=100, mask_percentage=0.10):
    """
    Splits a dataset into training and test sets, masking a portion of test set entries.
    
    Parameters:
    - df (pd.DataFrame): The dataset to split.
    - reviewer_col (str): The column name containing reviewer IDs.
    - random_state (int): The random state for reproducibility.
    - test_size (int): The number of reviewers to sample for the test set.
    - mask_percentage (float): The percentage of beers to mask for each reviewer in the test set.

    Returns:
    - df_train (pd.DataFrame): The training set.
    - df_test_masked (pd.DataFrame): The test set with masked entries.
    """
    # Randomly sample reviewers
    sampled_reviewers = df[reviewer_col].sample(n=test_size, random_state=random_state)
    
    # Get reviews from the sampled reviewers
    df_test = df[df[reviewer_col].isin(sampled_reviewers)]
    
    # Group by reviewer to get each user's beers
    df_test_grouped = df_test.groupby(reviewer_col)
    
    # Randomly mask a percentage of beers for each reviewer
    test_set_masked = []
    for reviewer, group in df_test_grouped:
        # Calculate how many beers to mask
        num_to_mask = max(int(len(group) * mask_percentage), 1)
        
        # Sample the calculated number of beers
        masked_group = group.sample(n=num_to_mask, random_state=random_state)
        test_set_masked.append(masked_group)
    
    # Combine masked reviews into a single DataFrame
    df_test_masked = pd.concat(test_set_masked)
    
    # Remove masked reviews from the training data
    df_train = df.drop(df_test_masked.index)
    
    # Display dataset summaries
    print("\n### Dataset Summary ###")
    print(f"Total reviewers sampled: {len(sampled_reviewers)}")
    print(f"Training set size: {df_train.shape}")
    print(f"Test set size: {df_test_masked.shape}")
    
    return df_train, df_test_masked

In [3]:
# Load data and preprocess
df = pd.read_pickle('encoded_beers_SBERT.pkl')

df_filtered = preprocess_data(df)
del df

Size after drop_duplicates:  1157819


/var/folders/ql/h53w_qjs60509nsf4jbr01c00000gn/T/ipykernel_38694/3114772741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['rating'] = pd.to_numeric(df_filtered['rating'], errors='coerce')  # Set erros to NaN


Size after drop rating NA:  1157807
Size after drop abv NA:  1154739


In [5]:
df_filtered.head()

,id,name,brewery,subgenre,abv,location,rating,average_rating,reviewer,review_date,review_text,algorithm_rating,total_reviews,sbert_embedding
0,1,Wild Dog Pale Ale,Wild Dog (Tiemann Beer),American Pale Ale,5.2,🇯🇪Jersey,3.5,2.99,Jerseyislandbeer,"December 14, 2023",330ml can from Shoprite in Livingstone. At hom...,28.0,11,"[0.037878353, 0.00593541, 0.0062317043, -0.011..."
1,2,Wild Dog Pale Ale,Wild Dog (Tiemann Beer),American Pale Ale,5.2,"🇬🇧Ipswich, England",3.2,2.99,Grumbo,"February 28, 2022","18/2/2022. Can sample courtesy of fonefan, che...",28.0,11,"[-0.037820198, -0.044825517, 0.07764052, 0.065..."
2,3,Wild Dog Pale Ale,Wild Dog (Tiemann Beer),American Pale Ale,5.2,"🇸🇪Tyresö, Sweden",3.5,2.99,omhper,"February 19, 2022","--Sample, thanks fonefan! -- Hazy deep golden,...",28.0,11,"[0.056960188, -0.00059301173, 0.11057871, 0.02..."
3,4,Wild Dog Pale Ale,Wild Dog (Tiemann Beer),American Pale Ale,5.2,"🇫🇮Vasa, Finland",2.8,2.99,oh6gdx,"January 31, 2022","Panda from a can, thanks fonefan!. Golden colo...",28.0,11,"[0.003549767, -0.010705345, 0.02083684, 0.0106..."
4,6,Wild Dog Pale Ale,Wild Dog (Tiemann Beer),American Pale Ale,5.2,"🇩🇰Haderslev, Denmark",2.6,2.99,martin00sr,"January 8, 2022","Can @Ulfborg. Cloudy amber, white head. Malty ...",28.0,11,"[-0.01005388, -0.02942978, 0.0016338513, 0.017..."


In [4]:
df_train, df_test_masked = create_test_train(df_filtered)


### Dataset Summary ###
Total reviewers sampled: 100
Training set size: (1149910, 14)
Test set size: (4829, 14)


In [5]:
# Create dataframe for retrieving beer information
beer_info = df_filtered[['name', 'abv', 'subgenre']]

# Drop duplicate rows based on the 'name' column (i.e. beers)
beer_info = beer_info.drop_duplicates(subset='name')

beer_info.set_index('name', inplace=True)

### Create locality-sensitive hashing (LSH)

In [6]:
# Initialize a variable to store the model
sbert_model = None

def encode_sbert(query, model_name='sentence-transformers/all-MiniLM-L6-v2'):
    """
    Encodes a query using SBERT. Loads the model if not already loaded.
    
    Parameters:
        query (str or list of str): The query or list of queries to encode.
        model_name (str): The name of the SBERT model to load (default is 'all-MiniLM-L6-v2').
    
    Returns:
        numpy.ndarray: The embedding(s) for the input query/queries.
    """
    global sbert_model  # Use the global variable to store the model
    
    # Load the model if it's not already loaded
    if sbert_model is None:
        sbert_model = SentenceTransformer(model_name)
    
    # Encode the query and return the embeddings
    return sbert_model.encode(query)

def generate_hyperplanes(dim, num_hash_functions):
    """
    Generate random hyperplanes for hash functions.
    
    Parameters:
    - dim: Dimensionality of the embeddings.
    - num_hash_functions: Number of hash functions per table.
    
    Returns:
    - A matrix of shape (num_hash_functions, dim) where each row is a hyperplane.
    """
    return np.random.randn(num_hash_functions, dim)

def hash_vectors(vectors, hyperplanes):
    """
    Hash a batch of vectors using a set of hyperplanes.

    Parameters:
    - vectors: Input vectors (2D array of shape [n_samples, d]).
    - hyperplanes: Matrix of hyperplanes (2D array of shape [k, d]).

    Returns:
    - A matrix of binary hash values (shape [n_samples, k]).
    """
    # Compute dot products and return binary hash values
    return (np.dot(vectors, hyperplanes.T) > 0).astype(int)

class LSHVectorized:
    def __init__(self, d, k, L):
        """
        Initialize the LSH scheme with vectorized support.

        Parameters:
        - d: Dimensionality of the input vectors.
        - k: Number of hash functions per table.
        - L: Number of hash tables.
        """
        self.L = L
        self.tables = [defaultdict(list) for _ in range(L)]
        self.hyperplanes = [generate_hyperplanes(d, k) for _ in range(L)]

    def add_vectors(self, vectors, identifiers):
        """
        Add a batch of vectors to the LSH index.

        Parameters:
        - vectors: Input vectors (2D array of shape [n_samples, d]).
        - identifiers: A list of unique identifiers for the vectors.
        """
        for table, hyperplanes in zip(self.tables, self.hyperplanes):
            # Compute hash values for all vectors at once
            hash_values = hash_vectors(vectors, hyperplanes)
            
            # Convert binary hash values to tuples for dictionary keys
            hash_keys = [tuple(h) for h in hash_values]
            
            # Add vectors to their corresponding buckets
            for identifier, key in zip(identifiers, hash_keys):
                table[key].append(identifier)

    def query(self, vectors):
        """
        Query the LSH index to find similar items for a batch of vectors.

        Parameters:
        - vectors: Query vectors (2D array of shape [n_samples, d]).

        Returns:
        - A list of sets, where each set contains the candidates for a query vector.
        """
        candidates = [set() for _ in range(len(vectors))]
        for table, hyperplanes in zip(self.tables, self.hyperplanes):
            # Compute hash values for all query vectors
            hash_values = hash_vectors(vectors, hyperplanes)
            
            # Convert binary hash values to tuples for dictionary keys
            hash_keys = [tuple(h) for h in hash_values]
            
            # Retrieve candidates for each query
            for i, key in enumerate(hash_keys):
                candidates[i].update(table.get(key, []))
        return candidates

In [7]:
vectors = np.vstack(df_train["sbert_embedding"].values)  # Combine embeddings into a 2D array
identifiers = df_train.index.tolist()  # Use review IDs as identifiers

In [8]:
## Run LSH ##
# Initialize LSH scheme
d = 384
k = 14 
L = 7

lsh = LSHVectorized(d, k, L)

In [9]:
# Add vectors to the LSH index
lsh.add_vectors(vectors, identifiers)

### Create Collaborative Filtering (CF)

In [10]:

def predict_ratings_user_based(user_item_matrix, similarity_matrix):
   
    """
    this function predicts the ratings for the user_item_matrix using the similarity_matrix
    

    Parameters: 
    
    - user_item_matrix (DataFrame): User-item matrix with ratings centered around the user mean.
    - similarity_matrix (DataFrame): User-user similarity matrix.
    
    Returns:
        - pred (DataFrame): Predicted ratings for all user-item pairs.
    
    """


    # Compute predictions
    similarity_sum = np.abs(similarity_matrix).sum(axis=1)[:, None]
    pred = np.dot(similarity_matrix, user_item_matrix) / (similarity_sum + 1e-8)



    return pred


def collaborative_filtering(df):
    
    """
    Predicts user ratings for items using user-based collaborative filtering with cosine similarity. 
    Preprocesses the input data to create a centered user-item matrix, computes user similarities, 
    and generates predicted ratings.

    Parameters:
    - df (DataFrame): Input data with 'reviewer', 'name', and 'rating' columns.

    Returns:
    - pr_df (DataFrame): Predicted ratings for all user-item pairs.

    """


    user_item_matrix = df.pivot_table(
    index="reviewer",     # Rows: Reviewers
    columns="name",       # Columns: Beer names
    values="rating",      # Values: Ratings
    fill_value=0          # Fill missing ratings with 0
    )
    

    #user_means = user_item_matrix.replace(0, np.nan).mean(axis=1).fillna(0).to_numpy()
    #print(user_means)
    #user_item_np = np.where(user_item_matrix != 0, user_item_matrix - user_means[:, None], 0)
    user_item_np = np.where(user_item_matrix != 0, (user_item_matrix - 3) / 2, 0)
    user_item_matrix = pd.DataFrame(user_item_np, index=user_item_matrix.index, columns=user_item_matrix.columns)


    # Compute cosine similarity
    cosine_similarity_matrix = cosine_similarity(user_item_matrix)
    
    # Predict ratings
    predicted_ratings = predict_ratings_user_based(user_item_matrix, cosine_similarity_matrix)

    df_out = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)
   
    return df_out


collab_df = collaborative_filtering(df_train)

In [11]:
# Caculate fraction of beers per user which is zero
zero_percentage = (collab_df == 0).mean(axis=1) * 100
zero_percentage.describe()

count    33531.000000
mean         4.749085
std         13.940269
min          0.034176
25%          0.128161
50%          0.375940
75%          1.768626
max        100.000000
dtype: float64

### Review Term Explanations:

In [12]:

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from sklearn.feature_extraction.text import CountVectorizer

# Define reference context for flavor-related words. The first 20 is from a aromatic kit used for sommeliers, the rest is ai-generated.
context_words = [
    "bitter", "sweet", "salt", "sour", "umami",
    "lemon", "grapefruit", "apple", "pear", "blackcurrant", "prune", "melon", 
    "banana", "acacia", "rose", "cut grass", "hay", "bay leaf", "thyme", 
    "tomato", "pepper", "nutmeg", "clove", "bread", "butter", "vanilla", 
    "hazelnut", "toast", "malt", "caramel", "honey", "coffee", "licorice",
    "pine", "grass", "resin", "floral", "perfume", "incense", "cinnamon",
    "ginger", "anise", "nut", "almond", "walnut", "chestnut", "peanut",
    "soy", "mushroom", "earth", "dust", "wood", "barnyard", "horse",
    "wet", "dry", "metallic", "sulfur", "fish", "cheese", "butter",
    "cream", "leather", "silk", "rubber", "barnyard", "ammonia",
    "rotten", "acid"
]
custom_stop_words = ["beer", "beers", "bottle", "taste", "nice", "aroma", "like", "good", "great", "head", "flavor", "flavors", "flavour", "flavours", "brew", "can"]
context_embeddings = encode_sbert(context_words)

# Function to filter terms dynamically
def is_flavor_related(term, context_embeddings, threshold=0.35):
    term_embedding = sbert_model.encode([term])[0]
    cosine_similarity = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
    max_similarity = max(cosine_similarity(term_embedding, context) for context in context_embeddings)
    return max_similarity > threshold

def plot_bucket(bucket_vectors, cluster_labels, perplexity=30, n_iter=5000, learning_rate=200):
    """
    Visualizes differences within an LSH bucket using t-SNE with configurable parameters.
    
    Args:
        bucket_vectors (np.ndarray): High-dimensional vectors of beers in the bucket.
        cluster_labels (np.ndarray): Cluster labels assigned to each vector.
        subgenres (np.ndarray): Subgenre or categorical labels for each beer.
        perplexity (int): The t-SNE perplexity parameter, balancing local/global data views.
        n_iter (int): Number of iterations for t-SNE optimization.
        learning_rate (float): Learning rate for t-SNE optimization.
    """
    # t-SNE reducer with tuned parameters
    reducer = TSNE(
        n_components=2, 
        random_state=42, 
        perplexity=perplexity, 
        n_iter=n_iter, 
        learning_rate=learning_rate
    )
    reduced_vectors = reducer.fit_transform(bucket_vectors)
    
    # Plotting
    plt.figure(figsize=(16, 10))
    scatter = plt.scatter(
        reduced_vectors[:, 0],
        reduced_vectors[:, 1],
        c=cluster_labels,
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Cluster Label')
    plt.title(f"t-SNE Visualization (Perplexity={perplexity}, n_iter={n_iter}, LR={learning_rate})")
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.grid(alpha=0.3)
    plt.show()

def getThemes(df_filtered, beer_name, query_embedding, limit=100):
    # Initialize CountVectorizer with custom stopwords
    default_stop_words = CountVectorizer(stop_words='english').get_stop_words()
    all_stop_words = list(set(default_stop_words).union(custom_stop_words))

    # Pass the combined stop words to CountVectorizer
    vectorizer = CountVectorizer(max_features=100, stop_words=all_stop_words, token_pattern=r'\b[a-zA-Z]{2,}\b')
    df_beer = df_filtered[df_filtered["name"] == beer_name]
    # Extract top terms from cluster reviews
    term_matrix = vectorizer.fit_transform(df_beer["review_text"])
    terms = vectorizer.get_feature_names_out()
    term_counts = np.array(term_matrix.sum(axis=0)).flatten()
    top_terms = [terms[i] for i in term_counts.argsort()[-limit:]]  # Top 5 terms
    filtered_top_terms = [term for term in top_terms if is_flavor_related(term, context_embeddings)]
    
    # Return the terms most similar to the query
    term_embeddings = np.vstack([encode_sbert(term) for term in filtered_top_terms])
    
    # Calculate cosine similarity between query and terms
    similarities = cosine_similarity(query_embedding, term_embeddings)[0]
    
    # Create a DataFrame to store terms and their similarities
    term_similarity_df = pd.DataFrame({
        'term': filtered_top_terms,
        'similarity': similarities
    })
    
    # Sort terms by similarity to the query
    term_similarity_df = term_similarity_df.sort_values(by='similarity', ascending=False)
    
    # Return the top similar themes
    return term_similarity_df['term'].head(10).tolist()


### K-means clustering of word vectors in buckets

In [13]:
def kmeans_bucket(bucket_vectors, bucket_data, embedded_query, n_clusters = 15):
    # Perform clustering on bucket vectors
    kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=7)
    cluster_labels = kmeans.fit_predict(bucket_vectors)
    
    # Assign query to the nearest cluster
    query_cluster = kmeans.predict(embedded_query)[0]

    # Filter beers in the same cluster as the query
    cluster_indices = np.where(cluster_labels == query_cluster)[0]
    cluster_vectors = bucket_vectors[cluster_indices]
    cluster_beers = bucket_data.iloc[cluster_indices]
    
    # Compute similarities within the selected cluster
    sims = cosine_similarity(embedded_query, cluster_vectors)[0]

## Make reccomendations

In [14]:
def recommend_beer_B(query_embedding, df_train, user_name, beer_info, abv_desired=None, style_desired=None, n_clusters=15):
    # Query the LSH index
    candidates = lsh.query(query_embedding)

    # Filter bucket vectors and metadata
    bucket_data = df_train[df_train["id"].isin(list(candidates[0]))]
    bucket_vectors = np.vstack(bucket_data["sbert_embedding"].to_numpy())
    
    # Extract subgenre information
    subgenres = bucket_data["subgenre"].values  # Adjust column name as necessary

    # Perform clustering on bucket vectors
    kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
    cluster_labels = kmeans.fit_predict(bucket_vectors)
    
    # Assign query to the nearest cluster
    query_cluster = kmeans.predict(query_embedding)[0]

    # Filter beers in the same cluster as the query
    cluster_indices = np.where(cluster_labels == query_cluster)[0]
    cluster_vectors = bucket_vectors[cluster_indices]
    cluster_beers = bucket_data.iloc[cluster_indices]
    
    # Compute similarities within the selected cluster
    sims = cosine_similarity(query_embedding, cluster_vectors)[0]

    # Perform collaborative filtering
    predcicted_rating_user = collab_df.loc[user_name]
    
    beer_LSH = pd.DataFrame({
        'similarity': sims,
        'beer': cluster_beers["name"].values,  # Adjust column name if necessary
    })
    
    LSH_score = beer_LSH.groupby('beer')['similarity'].mean()
    
    collab_filtering_scores = predcicted_rating_user[LSH_score.index.tolist()] # Get CF_score for user for the beers in cluster

    if abv_desired:
        # Penalise difference in abv by using a nonlinear function penalising greater differences more
        abv = beer_info.loc[LSH_score.index.tolist()]["abv"]

        alpha = 0.05
        if abv_desired == 0:
            abv_weight = -2 * abs(abv - abv_desired)
        else:
            abv_weight = -alpha * ((abv - abv_desired)**2) / (abv_desired**1.5 + 1)
    else:
        abv_weight = 0



    beta = 0.6 + 0.004*zero_percentage.loc[user_name] # Linear function to scale beta such that persons with few reviews rely more on LSH and vice versa
    
    # Add bonus for match in style
    style_bonus = np.zeros(len(LSH_score))
    if style_desired:
        relevant_styles = beer_info.loc[LSH_score.index.tolist()]["subgenre"]
        style_mask = relevant_styles == style_desired
        style_bonus[style_mask] = 0.05
    
    
    # Combine weights and scores
    weighted_score = (
        beta * LSH_score +
        (1-beta) * collab_filtering_scores +
        abv_weight +
        style_bonus
    )

    # Create final DataFrame
    beer_weighted_score = pd.DataFrame({
        'beer': LSH_score.index,
        'score': weighted_score,
        'abv': abv.values,
        'LSH_score': LSH_score.values,
        'LSH_score_weighted': LSH_score.values * beta,
        'collab_score': collab_filtering_scores.values,
        'collab_score_weighted': collab_filtering_scores.values * (1-beta),
        'abv_diff': abs(abv.values - abv_desired),
        'abv_weight': abv_weight.values,
        'Style bonus': style_bonus,
        'Weight beta': beta
    })
    
    # Remove index of weighted score and keep the beer name as a column
    beer_weighted_score.reset_index(drop=True, inplace=True)
    
    # Get the 10 beers with the highest weighted scores
    beer_weighted_score = beer_weighted_score.sort_values(by='score', ascending=False)
    beer_weighted_score['notes'] = ""
    # Apply getThemes to each beer in the DataFrame
    #beer_weighted_score['notes'] = beer_weighted_score['beer'].apply(lambda x: getThemes(df_train, x, query_embedding))
    #beer_weighted_score.loc[:9, 'notes'] = beer_weighted_score.loc[:9, 'beer'].apply(
    #    lambda x: getThemes(df_train, x, query_embedding)) # TODO Fix!

    return beer_weighted_score.sort_values(by='score', ascending=False)

    

In [35]:
# Create a query
test_query = "Light, refreshing bitter beer with a orange taste"
user_name = "Jerseyislandbeer"
query_embedding = encode_sbert(test_query).reshape(1, -1)
beer_recommendations= recommend_beer_B(query_embedding, df_train, user_name, beer_info, abv_desired=7)

print("Top 5 recommended beers:")
# Set max column width to display full array
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
display(beer_recommendations.head(10))

Top 5 recommended beers:


,beer,score,abv,LSH_score,LSH_score_weighted,collab_score,collab_score_weighted,abv_diff,abv_weight,Style bonus,Weight beta,notes
639,Nørrebro Pacific Summer Ale (Økologisk),0.460435,5.6,0.777055,0.466313,-0.002144,-0.000857,1.4,-0.005020,0.0,0.600103,
424,Insel-Brauerei Insel Saison,0.455709,5.5,0.772663,0.463677,-0.005514,-0.002205,1.5,-0.005763,0.0,0.600103,
508,Legenda Brutal Bitter IPA,0.450131,8.2,0.756718,0.454108,-0.000722,-0.000289,1.2,-0.003688,0.0,0.600103,
82,Baden Baden Witbier,0.445461,4.9,0.761292,0.456853,-0.000240,-0.000096,2.1,-0.011296,0.0,0.600103,
161,Browar Brodacz Mózg,0.437152,5.7,0.735744,0.441522,-0.000103,-0.000041,1.3,-0.004329,0.0,0.600103,
529,MONYO American Beauty APA,0.436200,5.6,0.737444,0.442542,-0.003306,-0.001322,1.4,-0.005020,0.0,0.600103,
292,Fanø Vestkyst,0.434280,5.7,0.730639,0.438458,0.000377,0.000151,1.3,-0.004329,0.0,0.600103,
92,Basqueland Saison (Lasai'son),0.433805,5.5,0.732640,0.439659,-0.000227,-0.000091,1.5,-0.005763,0.0,0.600103,
244,Det Lille Bryggeri Humlemord Tears From Heaven,0.425823,7.4,0.710667,0.426473,-0.000603,-0.000241,0.4,-0.000410,0.0,0.600103,
451,Kaapse Tess,0.425089,5.0,0.726712,0.436101,-0.001917,-0.000767,2.0,-0.010246,0.0,0.600103,


In [46]:
num_true = 0
num_false = 0
for i in range(100):
    review_row = df_test_masked.iloc[i]
    real_beer = review_row["name"]
    query = review_row["sbert_embedding"].reshape(1, -1)
    user = review_row["reviewer"]
    abv_desired = review_row["abv"]
    style_desired = review_row["subgenre"]

    beer_recommendations= recommend_beer_B(query_embedding=query, df_train=df_train, user_name=user, beer_info=beer_info, abv_desired=abv_desired, style_desired=style_desired)
    if real_beer in beer_recommendations["beer"]:
        num_true += 1
    else:
        num_false +=1


In [47]:
num_true

0

In [48]:
df_test_masked

,id,name,brewery,subgenre,abv,location,rating,average_rating,reviewer,review_date,review_text,algorithm_rating,total_reviews,sbert_embedding
2187929,1520794,Founders All Day IPA,Founders Brewing Company (Mahou San Miguel),Session IPA,4.7,"🇩🇰randers, Denmark",3.7,3.59,20107589,"August 12, 2016","pour pale without head. piny, light citrus, ar...",90.0,2303,"[0.0093215825, -0.04165539, 0.031013042, 0.068..."
479957,581970,Westvleteren 12,Abdij St. Sixtus - Westvleteren,Ice Cider/Perry,10.2,"🇩🇰randers, Denmark",3.7,4.42,20107589,"September 27, 2013","apples, caramel, roasted malt, liqurish aromas...",100.0,3705,"[0.016198197, -0.039563965, 0.054247003, 0.086..."
2841653,2264997,Dupont Monk's Stout,Brasserie Dupont,Stout,5.2,"🇩🇰randers, Denmark",3.5,3.30,20107589,"August 15, 2013","coffee, smoke and bread aromas, fine black app...",51.0,719,"[-0.0059644626, -0.052805316, 0.05065915, 0.08..."
289833,337086,Thisted Seattle,Thisted Bryghus,Stout,6.5,"🇩🇰randers, Denmark",2.9,3.42,20107589,"February 8, 2015","clear coffee aroma, with vanilla notes. appear...",71.0,125,"[0.027789045, -0.07606227, 0.057118244, 0.1007..."
549186,658714,Three Floyds Zombie Dust,Three Floyds Brewing Company,Anglo-American Ales,6.2,"🇩🇰randers, Denmark",4.1,4.11,20107589,"May 5, 2015","pour pale golden, with a fine white head. ligh...",100.0,1475,"[0.025810374, -0.062446844, 0.07448743, 0.0553..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101968,325014,Tucher Dunkles Hefe Weizen,Tucher Bräu (Radeberger Group),Dunkelweizen,5.2,"🇺🇸Princeton, United States",3.6,3.26,wnhay,"September 19, 2005",Poured orange brown with a good tan head. Aro...,48.0,454,"[-0.023349693, 0.0337615, 0.030568456, 0.07432..."
2810991,2229220,Saison Dupont,Brasserie Dupont,Saison,6.5,"🇺🇸Princeton, United States",3.6,3.82,wnhay,"November 30, 2004",This beer had a bitter fruit taste which I kin...,98.0,2830,"[-0.025270637, -0.013938596, -0.03224327, 0.04..."
2488245,1859848,La Chouffe Blonde,Brasserie d'Achouffe (Duvel Moortgat),Belgian Strong Ale,8.0,"🇺🇸Princeton, United States",3.2,3.76,wnhay,"December 20, 2004",Sampled this. I thought the taste and aroma w...,97.0,2728,"[-0.03302289, -0.030031314, 0.012916914, -0.02..."
795399,913821,Harvest Moon Full Moon Pale Ale,Harvest Moon Brewery (NJ),English Pale Ale,5.6,"🇺🇸Princeton, United States",3.5,3.38,wnhay,"April 2, 2006",Tap at the brew pub. Yellow/golden color with...,71.0,40,"[0.067461774, 0.02529345, -0.025659995, 0.0234..."


# TESTTEST

In [20]:
def recommend_beer(query_embedding, df_train, user_name, abv_desired, n_clusters=15):    
    # Query the LSH index
    candidates = lsh.query(query_embedding)

    # Filter bucket vectors and metadata
    bucket_data = df_train[df_train["id"].isin(list(candidates[0]))]
    bucket_vectors = np.vstack(bucket_data["sbert_embedding"].to_numpy())
    
    # Extract subgenre information
    subgenres = bucket_data["subgenre"].values  # Adjust column name as necessary
    
    # Perform clustering on bucket vectors
    kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
    cluster_labels = kmeans.fit_predict(bucket_vectors)
    
    # Assign query to the nearest cluster
    query_cluster = kmeans.predict(query_embedding)[0]
    
    perplexities = [50]
    n_iters = [10000]
    learning_rates = [100, 200]
    
    param_combinations = [(p, n, lr) for p in perplexities for n in n_iters for lr in learning_rates]
    
    #for perplexity, n_iter, learning_rate in param_combinations:
    #    plot_bucket(bucket_vectors, cluster_labels, subgenres, perplexity, n_iter, learning_rate)
        
    # Filter beers in the same cluster as the query
    cluster_indices = np.where(cluster_labels == query_cluster)[0]
    cluster_vectors = bucket_vectors[cluster_indices]
    cluster_beers = bucket_data.iloc[cluster_indices]
    
    # Compute similarities within the selected cluster
    sims = cosine_similarity(query_embedding, cluster_vectors)[0]

    # Perform collaborative filtering
    predcicted_rating_user = collab_df.loc[user_name]
    
    beer_LSH = pd.DataFrame({
        'similarity': sims,
        'beer': cluster_beers["name"].values,  # Adjust column name if necessary
    })
    
    LSH_score = beer_LSH.groupby('beer')['similarity'].mean()
    
    collab_filtering_scores = predcicted_rating_user[LSH_score.index.tolist()]
    
    # Penalise difference in abv by using a nonlinear function penalising greater differences more
    abv = beer_info.loc[LSH_score.index.tolist()]["abv"]

    alpha = 0.05
    if abv_desired == 0:
        abv_weight = -2 * abs(abv - abv_desired)
    else:
        abv_weight = -alpha * ((abv - abv_desired)**2) / (abv_desired**1.5 + 1)
    

    # Combine weights and scores
    weighted_score = (
        0.85 * LSH_score +
        0.15 * collab_filtering_scores +
        abv_weight
    )
    
    # Create final DataFrame
    beer_weighted_score = pd.DataFrame({
        'beer': LSH_score.index,
        'score': weighted_score,
        'abv': abv.values,
        'LSH_score': LSH_score.values,
        'LSH_score_weighted': LSH_score.values * 0.85,
        'collab_score': collab_filtering_scores.values,
        'collab_score_weighted': collab_filtering_scores.values * 0.15,
        'abv_diff': abs(abv.values - abv_desired),
        'abv_weight': abv_weight.values
    })
    
    # Remove index of weighted score and keep the beer name as a column
    beer_weighted_score.reset_index(drop=True, inplace=True)
    
    # Get the 10 beers with the highest weighted scores
    beer_weighted_score = beer_weighted_score.sort_values(by='score', ascending=False).head(10)
    
    # Apply getThemes to each beer in the DataFrame
    beer_weighted_score['notes'] = beer_weighted_score['beer'].apply(lambda x: getThemes(df_train, x, query_embedding))


    return beer_weighted_score.sort_values(by='score', ascending=False)
    

# Create a query
test_query = "Light, refreshing bitter beer with a orange taste"
user_name = "Jerseyislandbeer"
query_embedding = encode_sbert(test_query).reshape(1, -1)
beer_recommendations= recommend_beer(query_embedding, df_train, user_name, 7)

print("Top 5 recommended beers:")
# Set max column width to display full array
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
display(beer_recommendations.head(10))


Top 5 recommended beers:


,beer,score,abv,LSH_score,LSH_score_weighted,collab_score,collab_score_weighted,abv_diff,abv_weight,notes
639,Nørrebro Pacific Summer Ale (Økologisk),0.655155,5.6,0.777055,0.660497,-0.002144,-0.000322,1.4,-0.005020,"[orange, oranges, bitter, citrus, citrusy, sweetness, ale, bitterness, grapefruit, fruitiness]"
424,Insel-Brauerei Insel Saison,0.650174,5.5,0.772663,0.656764,-0.005514,-0.000827,1.5,-0.005763,"[orange, tasting, bitter, citrus, sweetness, ale, wine, bitterness, sour, fruit]"
508,Legenda Brutal Bitter IPA,0.639414,8.2,0.756718,0.643210,-0.000722,-0.000108,1.2,-0.003688,"[orange, bitter, citrus, citrusy, sweetness, ipa, alcohol, bitterness, grape, grapefruit]"
82,Baden Baden Witbier,0.635766,4.9,0.761292,0.647098,-0.000240,-0.000036,2.1,-0.011296,"[orange, oranges, bitter, citrus, sweetness, ale, bitterness, ales, drinkability, sour]"
529,MONYO American Beauty APA,0.621311,5.6,0.737444,0.626828,-0.003306,-0.000496,1.4,-0.005020,"[orange, tasting, bitter, citrus, citrusy, sweetness, ale, bitterness, grapefruit, sour]"
161,Browar Brodacz Mózg,0.621038,5.7,0.735744,0.625382,-0.000103,-0.000015,1.3,-0.004329,"[orange, bitternes, bitter, citrus, citrusy, ale, bitterness, ales, sour, drinkable]"
92,Basqueland Saison (Lasai'son),0.616947,5.5,0.732640,0.622744,-0.000227,-0.000034,1.5,-0.005763,"[orange, bitter, citrus, sweetness, bitterness, beery, aromas, sour, fruit, spicy]"
292,Fanø Vestkyst,0.616771,5.7,0.730639,0.621043,0.000377,0.000056,1.3,-0.004329,"[orange, bitter, citrus, sweetness, ipa, ale, bitterness, grapefruit, fruit, spicy]"
451,Kaapse Tess,0.607172,5.0,0.726712,0.617705,-0.001917,-0.000288,2.0,-0.010246,"[orange, bitter, citrus, sweetness, wine, bitterness, grape, grapefruit, fruitiness, sour]"
653,Orca / Heidenpeters / Ale Mania Head in the Clouds,0.606479,3.9,0.742507,0.631131,-0.000244,-0.000037,3.1,-0.024615,"[orange, bitter, citrus, bitterness, grapefruit, sour, lager, phenolic, coffee, freshness]"


### Notes for improvement
Add better stop-words, flavor, flavour, flavors etc.


### Example:

### Evaluation settup